# Character-level Language Modeling with LSTMs

This notebook is adapted from [Keras' lstm_text_generation.py](https://github.com/fchollet/keras/blob/master/examples/lstm_text_generation.py).

Steps:

- Download a small text corpus and preprocess it.
- Extract a character vocabulary and use it to vectorize the text.
- Train an LSTM-based character level langague model.
- Use the trained model to sample random text with varying entropy levels.
- Implement greedy and beam-search deterministic decoders.


**Note**: fitting language models is compute intensive. It is recommended to do this notebook on a server with a GPU or powerful CPUs that you can leave running for several hours at once.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Loading some text data

Let's use some publicly available philosopy:

In [ ]:
from keras.utils.data_utils import get_file

URL = "https://s3.amazonaws.com/text-datasets/nietzsche.txt"

corpus_path = get_file('nietzsche.txt', origin=URL)
text = open(corpus_path).read().lower()
print('Corpus length: %d characters' % len(text))

In [ ]:
print(text[:600], "...")

In [ ]:
text = text.replace("\n", " ")
split = int(0.9 * len(text))
train_text = text[:split]
test_text = text[split:]

## Building a vocabulary of all possible symbols 

To simplifly things, we build a vocabulary by extracting the list all possible characters from the full datasets (train and validation).

In a more realistic setting we would need to take into account that the test data can hold symbols never seen in the training set. This issue is limited when we work at the character level though.

Let's build the list of all possible characters and sort it to assign a unique integer to each possible symbol in the corpus:

In [ ]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

`char_indices` is a mapping to from characters to integer identifiers:

In [ ]:
len(char_indices)

In [ ]:
sorted(char_indices.items())[:15]

`indices_char` holds the reverse mapping:

In [ ]:
len(indices_char)

In [ ]:
indices_char[52]

While not strictly required to build a language model, it's a good idea to have a look a the distribution of relative frequencies of each symbol in the corpus:

In [ ]:
from collections import Counter

counter = Counter(text)
chars, counts = zip(*counter.most_common())
indices = np.arange(len(counts))

plt.figure(figsize=(14, 3))
plt.bar(indices, counts, 0.8)
plt.xticks(indices, chars);

Let's cut the dataset into fake sentences at random with some overlap. Instead of cutting at random we could use a English specific sentence tokenizer. This is explained at the end of this notebook. In the mean time random substring will be good enough to train a first language model.

In [ ]:
max_length = 40
step = 3


def make_sequences(text, max_length=max_length, step=step):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - max_length, step):
        sequences.append(text[i: i + max_length])
        next_chars.append(text[i + max_length])
    return sequences, next_chars    


sequences, next_chars = make_sequences(train_text)
sequences_test, next_chars_test = make_sequences(test_text, step=10)

print('nb train sequences:', len(sequences))
print('nb test sequences:', len(sequences_test))

Let's shuffle the sequences to break some of the dependencies:

In [ ]:
from sklearn.utils import shuffle

sequences, next_chars = shuffle(sequences, next_chars,
                                random_state=42)

In [ ]:
sequences[0]

In [ ]:
next_chars[0]

## Converting the training data to one-hot vectors

Unfortunately the LSTM implementation in Keras does not (yet?) accept integer indices to slice columns from an input embedding by it-self. Let's use one-hot encoding. This is slightly less space and time efficient than integer coding but should be good enough when using a small character level vocabulary.

**Exercise:** 

One hot encoded the training `data sequences` as `X` and `next_chars` as `y`:

In [ ]:
n_sequences = len(sequences)
n_sequences_test = len(sequences_test)
voc_size = len(chars)

X = np.zeros((n_sequences, max_length, voc_size),
             dtype=np.float32)
y = np.zeros((n_sequences, voc_size), dtype=np.float32)

X_test = np.zeros((n_sequences_test, max_length, voc_size),
                  dtype=np.float32)
y_test = np.zeros((n_sequences_test, voc_size), dtype=np.float32)

# TODO

In [ ]:
# %load solutions/language_model_one_hot_data.py

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X[0]

In [ ]:
y[0]

## Measuring perplexity

The NLP community measures the quality of probabilistic model using [perplexity](https://en.wikipedia.org/wiki/Perplexity).

In practice perplexity is just a base 2 exponentiation of nll:

$$perplexity_\theta = 2^{-\frac{1}{n} \sum_{i=1}^{n} log_2 (p_\theta(x_i))}$$

**Exercise**: implement a perplexity function with model predicted probabilities `y_pred` and `y_true` for the encoded ground truth:

In [ ]:
def perplexity(y_true, y_pred):
    """Compute the perplexity of model predictions.
    
    y_true is one-hot encoded ground truth.
    y_pred is predicted likelihoods for each class.
    
    2 ** -mean(log2(p))
    """
    # TODO
    return 1.

In [ ]:
# %load solutions/language_model_perplexity.py

In [ ]:
y_true = np.array([
    [0, 1, 0],
    [0, 0, 1],
    [0, 0, 1],
])

y_pred = np.array([
    [0.1, 0.9, 0.0],
    [0.1, 0.1, 0.8],
    [0.1, 0.2, 0.7],
])

perplexity(y_true, y_pred)

A perfect model has a minimal perplixity of 1.0 (negative log likelihood of 0.0):

In [ ]:
perplexity(y_true, y_true)

## Building recurrent model

Let's build a first model and train it on a very small subset of the data to check that it works as expected:

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam


model = Sequential()
model.add(LSTM(128, input_shape=(max_length, voc_size)))
model.add(Dense(voc_size, activation='softmax'))

optimizer = Adam(lr=0.001, clipnorm=10)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

Let's measure the perplexity of the randomly initialized model:

In [ ]:
def model_perplexity(model, X, y):
    predictions = model.predict(X, verbose=1)
    return perplexity(y, predictions)


model_perplexity(model, X_test, y_test)

Let's train the model for one epoch on a very small subset of the training set to check that it's well defined:

In [ ]:
small_train = slice(0, 100000, 40)

model.fit(X[small_train], y[small_train], validation_split=0.1,
          batch_size=128, nb_epoch=1)

In [ ]:
model_perplexity(model, X[small_train], y[small_train])

In [ ]:
model_perplexity(model, X_test, y_test)

## Training the model

## Sampling random text from the model

## Greedy deterministic decoding

## Beam search for deterministic decoding

## Better handling of sentence boundaries

To simplify things we used the lower case version of the text and we ignored any sentence boundaries. This prevents our model to learn when to stop generating characters. If we want to train a model that can start generating text at the beginning of a sentence and stop at the end of a sentence, we need to provide it with sentency boundary markers in the training set and use those special markers when sampling.

The following give an example of how to use NLTK to detect sentence boundaries in English text.

This could be used to insert an explicit "end_of_sentence" (EOS) symbol to mark separation between two consecutive sentences. This should make it possible to train a language model that explicitly generates complete sentences from start to end.

Use the following command (in a terminal) to install nltk before importing it in the notebook:

```
$ pip install nltk
```

In [ ]:
text_with_case = open(corpus_path).read().replace("\n", " ")

In [ ]:
import nltk

nltk.download('punkt')
from nltk.tokenize import sent_tokenize
sentences = sent_tokenize(text_with_case)

In [ ]:
plt.hist([len(s.split()) for s in sentences], bins=30);
plt.title('Distribution of sentence lengths')
plt.xlabel('Approximate number of words');

The first few sentences detected by NLTK are too short to be considered real sentences. Let's have a look at short sentences with at least 20 characters:

In [ ]:
sorted_sentences = sorted([s for s in sentences if len(s) > 20], key=len)
for s in sorted_sentences[:5]:
    print(s)

Some long sentences:

In [ ]:
for s in sorted_sentences[-3:]:
    print(s)

The NLTK sentence tokenizer seems to do a reasonable job despite the weird casing and '--' signs scattered around the text.

Note that here we use the original case information because it can help the NLTK sentence boundary detection model make better split decisions. Our text corpus is probably too small to train a good sentence aware language model though, especially with full case information. Using larger corpora such as a large collection of [public domain books](http://www.gutenberg.org/) or Wikipedia dumps. The NLTK toolkit also comes from [corpus loading utilities](http://www.nltk.org/book/ch02.html).

The following loads a selection of famous books from the Gutenberg project archive:

In [ ]:
import nltk

nltk.download('gutenberg')
book_selection_text = nltk.corpus.gutenberg.raw().replace("\n", " ")

In [ ]:
print(book_selection_text[:300])

In [ ]:
print("Book corpus length: %d characters" % len(book_selection_text))

Let's do an arbitrary split. Note the training set will have a majority of text that is not authored by the author(s) of the validation set:

In [ ]:
split = int(0.9 * len(book_selection_text))
book_selection_train = book_selection_text[:split]
book_selection_validation = book_selection_text[split:]

## Bonus exercises

- Adapt the previous language model to handle explicitly sentence boundaries with a special EOS character.
- Train a new model on the random sentences sampled from the the book selection corpus with full case information.
- Adapt the random sampling code to start sampling at the beginning of sentence and stop when the sentence ends.
- Train a deep GRU (e.g. two GRU layers instead of one LSTM) to see if you can improve the validation perplexity.
- Git clone the source code of the [Linux kernel](https://github.com/torvalds/linux) and train a C programming language model on it. Instead of sentence boundary markers, we could use source file boundary markers for this exercise.
- Try to increase the vocabulary size to 256 using a [Byte Pair Encoding](https://arxiv.org/abs/1508.07909) strategy.